<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Google GenAI SDK - Building an Evaluator-Optimizer Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai openinference-instrumentation-google-genai arize-phoenix-otel

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(
    auto_instrument=True, project_name="google-genai-evaluator-optimizer-agent"
)
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
from google import genai

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Evaluator Optimizer

In [ ]:
# --- 1. Instantiate the model(s) ---
# The global `genai` configuration (or the `client` created in a previous
# notebook cell) is already set up, so we can request models right away.
story_model = "gemini-2.0-flash-001"
critic_model = "gemini-2.0-flash-001"

# --- Configuration for iterative improvement ---
num_iterations = 2  # Set the number of critique/revision cycles


# --- 2. Generate the initial short story ---
@tracer.agent()
def generate_story(story_model, client):
    story_prompt = (
        "You are a creative short story writer.\n"
        "Write a brief, engaging story (3–4 paragraphs) about an unexpected "
        "adventure.\n"
        "Be imaginative but concise.\n"
        "User Input: "
    )
    story_response = client.models.generate_content(
        model=story_model,
        contents=story_prompt + input("Please enter a prompt or seed for your story: "),
    )
    # Initialize the story variable that will be updated in the loop
    current_story = story_response.text.strip()

    print("=== Initial Story ===\n")
    print(current_story)

    # --- 3. Iteratively Critique and Improve the story ---
    for i in range(num_iterations):
        print(f"\n--- Iteration {i+1} ---")

        # --- 3a. Critique the current story ---
        print("\nGenerating critique...")
        critic_prompt = (
            "You are a literary critic.\n"
            "Analyze the provided story for its strengths and weaknesses.\n"
            "Comment on plot, characters, and overall impact.\n"
            "Provide 2–3 specific suggestions for improvement.\n\n"
            f"Story:\n{current_story}"
        )
        critic_response = client.models.generate_content(model=critic_model, contents=critic_prompt)
        story_critique = critic_response.text.strip()

        # --- 3b. Display the critique ---
        print(f"\n=== Critique {i+1} ===\n")
        print(story_critique)

        # --- 3c. Improve the story based on the critique ---
        print("\nGenerating revision...")
        revision_prompt = (
            "You are a creative short story writer.\n"
            "Revise the following story based *only* on the critique provided.\n"
            "Focus on addressing the specific suggestions mentioned in the critique.\n"
            "Do not introduce significant new elements not prompted by the critique.\n\n"
            f"Critique:\n{story_critique}\n\n"
            f"Original Story:\n{current_story}\n\n"
            "Please produce an improved version of the story, addressing the suggestions.\n"
            "Output *only* the revised story."
        )
        revision_response = client.models.generate_content(
            model=story_model, contents=revision_prompt
        )
        # Update the current story with the improved version for the next iteration
        current_story = revision_response.text.strip()

        # --- 3d. Display the improved story for this iteration ---
        print(f"\n=== Improved Story {i+1} ===\n")
        print(current_story)

    print("\n--- Final Story after Iterations ---")
    print(current_story)
    return current_story


generate_story(story_model, client)